In [1]:
from importlib import reload
import sys
import os
sys.path.insert(0, './..')

In [3]:
import os
import sys
import numpy as np
import tensorflow as tf
import logging
from scipy import linalg

import dev_ori_sel_RF
from dev_ori_sel_RF import integrator_tf,\
    dynamics, network
from dev_ori_sel_RF.tools import misc

ModuleNotFoundError: No module named 'normalisation_constraints'